In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the EMNIST dataset (byclass for full A-Z, a-z, and digits)
ds_train, ds_test = tfds.load('emnist/byclass', split=['train', 'test'], as_supervised=True)


Dl Completed...: 0 url [00:00, ? url/s]
Extraction completed...: 100%|██████████| 4/4 [00:04<00:00,  1.04s/ file]


Dataset emnist downloaded and prepared to C:\Users\mridu\tensorflow_datasets\emnist\byclass\3.1.0. Subsequent calls will reuse this data.


In [4]:
def preprocess(image, label):
    # Rotate and normalize the images
    image = tf.image.rot90(image, k=3)  # Rotate 270 degrees
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0,1]
    return image, label

# Apply the preprocessing function to the datasets
ds_train = ds_train.map(preprocess).batch(32).shuffle(10000)
ds_test = ds_test.map(preprocess).batch(32)


In [28]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(62, activation='softmax')  # 36 classes for A-Z and 0-9
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [30]:
history = model.fit(ds_train, epochs=10,steps_per_epoch=1000)

Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - accuracy: 0.7855 - loss: 0.6365
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.8083 - loss: 0.5625
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.8178 - loss: 0.5179
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.8319 - loss: 0.4864
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.8360 - loss: 0.4704
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.8365 - loss: 0.4602
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.8422 - loss: 0.4517
Epoch 8/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.8403 - loss: 0.4498
Epoch 9/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.8434 - loss: 0.4316
Epoch 10/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.8415 - loss: 0.4385


In [31]:
test_loss, test_accuracy = model.evaluate(ds_test)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")


3636/3636 ━━━━━━━━━━━━━━━━━━━━ 25s 7ms/step - accuracy: 0.8494 - loss: 0.4181
Test accuracy: 84.93%


In [33]:

from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

In [34]:
OUTDIR = "./mnist_digits/model.weights.h5"
checkpoint_callback = ModelCheckpoint(
    OUTDIR, save_weights_only=True)

In [35]:
model.save('emnist_full_model.h5')